In [2]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt

In [3]:
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("ml-latest-small/movies.csv")
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
item = pd.read_csv("ml-latest-small/ratings.csv")
item.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df = pd.merge(df, item, on ='movieId')
df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


In [7]:
df.drop(columns =['timestamp', 'genres'], inplace = True)

In [8]:
ratings = pd.DataFrame(df.groupby('title').mean()['rating'])
ratings['freq'] = pd.DataFrame(df.groupby('title').count()['rating'])
ratings.head()

,rating,freq
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


In [9]:
mat = df.pivot_table(index ='userId', columns = 'title', values = 'rating')
mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [16]:
def get_movie(movie_name, number_of_items):
    movie_user_rating = mat[movie_name]
    similar_item = mat.corrwith(movie_user_rating)
    corr_item = pd.DataFrame(similar_item, columns = ['Correlation'])
    corr_item.dropna(inplace = True)
    
    corr_item = corr_item.join(ratings['freq'])
    corr_item = corr_item.join(ratings['rating'])
    prediction = corr_item[corr_item['freq'] > 100].sort_values('Correlation', ascending = False)
    
    prediction = prediction[['rating', 'Correlation']]
    ans =  prediction[1:].head(number_of_items)
    return ans;

In [17]:
print("Enter movie you liked: ")
movie_name = input()
print("Enter how many recommendations would you like: ")
number_of_items = int(input())
ans = get_movie(movie_name, number_of_items)
print(ans)

Enter movie you liked: 
Avengers, The (2012)
Enter how many recommendations would you like: 
5
                                             rating  Correlation
title                                                           
Men in Black (a.k.a. MIB) (1997)           3.487879     0.712532
True Lies (1994)                           3.497191     0.704203
Stargate (1994)                            3.375000     0.658420
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)  3.983051     0.642838
Mission: Impossible (1996)                 3.537037     0.604400
